# Multi-Agent Collaboration 
---

### What is Multi-Agent Collaboration?
Multi-Agent Collaboration refers to the process where multiple autonomous agents—each capable of independent decision-making—work together to achieve common or complementary objectives. This concept is widely used in fields like artificial intelligence, robotics, distributed computing, and simulation, and it involves several key aspects:

- **Effective Communication and Coordination**:
Agents exchange information and align their actions to collectively achieve a goal, ensuring that tasks are organized and synchronized.

- **Autonomous, Distributed Decision-Making**:
Each agent operates independently, making local decisions while contributing to a broader strategy, which enhances flexibility and fault tolerance.

- **Adaptive Task Specialization**:
Agents focus on specific roles or subtasks based on their capabilities, and they adjust their strategies through iterative feedback, leading to improved overall performance.


### Key Advantages
- **Efficiency Through Task Specialization**:
By assigning specific roles to different agents, the system can handle complex tasks in parallel. This specialization allows each agent to focus on its area of expertise, resulting in faster and more effective problem-solving.

- **Scalability and Flexibility**:
AutoGen's structured communication and dynamic task allocation enable the system to scale easily. It can adapt to varying project complexities by adding or reassigning agents as needed, ensuring that the collaboration remains robust even as demands change.

- **Enhanced Iterative Refinement**:
The framework’s built-in feedback loops and iterative dialogue facilitate continuous improvement. Agents can refine their outputs based on real-time feedback, leading to more accurate and cohesive final results.

**Reference**
- [AutoGen paper: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Multi-Agent Collabration Concept](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#network) 
- [LangChain `create_react_agent` built-in function](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) 

In [ ]:
import os
from dotenv import load_dotenv
from azure_genai_utils.tracer import get_langchain_api_key, set_langsmith

load_dotenv(override=True)

# If you want to trace your RAG API calls, please set the tracing=True. You need to have a valid Langchain API key.
langchain_key, has_langchain_key = get_langchain_api_key()
set_langsmith("[RAG Innv Lab] 1_Agentic-Design-Pattern", tracing=False)

azure_openai_chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

<br>

## 🧪 Step 1. Test and Construct each module
---

### Define your LLM
This hands-on only uses the `gpt-4o-mini`, but you can utilize multiple models in the pipeline.

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=azure_openai_chat_deployment_name, temperature=0)

### Tools

Before building the entire the graph pipeline, we will test and construct each module separately.

- **Researcher**
- **Coder**

In [ ]:
from azure_genai_utils.tools import BingSearch
from langchain_experimental.tools import PythonREPLTool

WEB_SEARCH_FORMAT_OUTPUT = True

web_search_tool = BingSearch(
    max_results=3,
    locale="en-US",
    include_news=False,
    include_entity=False,
    format_output=WEB_SEARCH_FORMAT_OUTPUT,
)

python_repl_tool = PythonREPLTool()

In [ ]:
web_search_tool.invoke("Where is Seoul?")

In [ ]:
python_repl_tool.invoke("print('Hello, World!')")

### ReAct agent test (Not required. Just for testing)

In [ ]:
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(llm, tools=[web_search_tool])
research_agent.invoke({"messages": "Where is Seoul?"})

In [ ]:
coder_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=None)
coder_agent.invoke({"messages": [("user", "print 'Hello, World!'")]})

<br>

## 🧪 Step 2. Define the Graph
---

### State Definition

- `messages`: Messages to be passed between agents
- `sender`: The sender of the next message

In [ ]:
import operator
from typing import Sequence, Annotated
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: Annotated[str, "The sender of the last message"]

### Create Agents

Create agents that perform sub-tasks.
- `Researcher`: Researches the topic
- `Coder`: Codes the solution

#### Researcher Agent

In [ ]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n\n{suffix}"
    )

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Create a Research Agent
research_agent = create_react_agent(
    llm,
    tools=[web_search_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)


# Research Agent Node definition
def research_node(state: AgentState) -> AgentState:

    result = research_agent.invoke(state)

    # Convert the last message to HumanMessage
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )

    return {"messages": [last_message], "sender": "researcher"}

Not required, but it is good to test the agent before creating the graph workflow.

In [ ]:
research_node({"messages": "Where is Seoul?", "sender": ""})

#### Coder (Chart Generator) Agent

In [ ]:
from langchain_core.prompts import load_prompt

code_system_prompt = load_prompt("../../../prompts/code-system-prompt-kr.yaml").format()
chart_agent = create_react_agent(
    llm,
    tools=[python_repl_tool],
    prompt=make_system_prompt(code_system_prompt),
)


def chart_node(state: AgentState) -> AgentState:

    result = chart_agent.invoke(state)

    last_message = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    return {
        # share internal message history of chart agent with other agents
        "messages": [last_message],
        "sender": "chart_generator",
    }

### Construct the Graph

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver


def router(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return "continue"


workflow = StateGraph(AgentState)

# Node definition
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

# Edge definition
workflow.add_conditional_edges(
    "researcher",
    router,
    {"continue": "chart_generator", END: END},
)
workflow.add_conditional_edges(
    "chart_generator",
    router,
    {"continue": "researcher", END: END},
)

workflow.add_edge(START, "researcher")

app = workflow.compile(checkpointer=MemorySaver())

### Visualize the graph

In [ ]:
from azure_genai_utils.graphs import visualize_langgraph

visualize_langgraph(app, xray=True)

<br>

## 🧪 Step 3. Execute the Graph
---

### Execute the graph

In [ ]:
from langchain_core.runnables import RunnableConfig
from azure_genai_utils.messages import stream_graph, invoke_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

inputs = {
    "messages": [
        HumanMessage(
            content="Please draw a graph of Korea's population growth rate over the past 10 years."
        )
    ],
}

invoke_graph(app, inputs, config, node_names=["researcher", "chart_generator", "agent"])